<a href="https://colab.research.google.com/github/pabloac31/TFG/blob/master/ImageNet_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/pabloac31/TFG.git
%cd TFG

Cloning into 'TFG'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 253 (delta 41), reused 17 (delta 8), pack-reused 183
Receiving objects: 100% (253/253), 217.42 MiB | 11.79 MiB/s, done.
Resolving deltas: 100% (134/134), done.
Checking out files: 100% (30/30), done.
/content/TFG


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
from utils import *
from adversarial_attacks import *
import torchvision.models as models
import json

In [3]:
# Define what device we are using
use_cuda=True
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  True


In [0]:
class_idx = json.load(open("imagenet_class_index.json"))

In [0]:
model = models.resnet50(pretrained=True)

In [0]:
def image_loader(img_name):
  """load image, returns CUDA tensor"""
  transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])

  img = Image.open(img_name)

  return transform(img)

In [0]:
img = torch.unsqueeze(image_loader('data/panda.jpg'), dim=0)

In [86]:
model.eval()
img.requires_grad = True

output = model(img)
pred = output.max(1, keepdim=True)[1]
print('Input image classified as', class_idx[str(pred.item())][1], end='')
percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
print(' with a {:.2f}% confidence'.format(percentage[pred].item()))

Input image classified as giant_panda with a 99.68% confidence


In [0]:
adv_x, pert_x = fgsm(model, img, label, output, epsilon=0.1, clip=True)

In [88]:
output = model(adv_x)
pred = output.max(1, keepdim=True)[1]
print('Adversarial image classified as', class_idx[str(pred.item())][1], end='')
percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
print(' with a {:.2f}% confidence'.format(percentage[pred].item()))

Adversarial image classified as giant_panda with a 79.02% confidence
